# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
pip install dotenv


  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
#HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(OPENAI_API_KEY)

In [ ]:
pip install pandas

In [98]:
import os

folder_path = r"C:\Users\Huawei\Desktop\ironhack\week 7\Day 1\lab-chains-in-langchain\data"
print(os.listdir(folder_path))


['Data.csv']


In [99]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Huawei\Desktop\ironhack\week 7\Day 1\lab-chains-in-langchain\data\Data.csv")
df.head()


,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
pip install langchain_community

In [ ]:
pip install langchain_openai

In [106]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [107]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [108]:

prompt = ChatPromptTemplate.from_template(
    "Give me a summary of customer feedback for the following product: {product_name}"
)


In [109]:

chain = LLMChain(llm=llm, prompt=prompt)

In [110]:
product ="Luxury Air Mattress"#Select a product type to be describe
chain.run(product)

'Overall, customers have provided positive feedback for the Luxury Air Mattress. Many customers have praised the comfort and quality of the mattress, noting that it provides a luxurious sleeping experience comparable to traditional mattresses. Customers also appreciate the durability and easy setup of the air mattress. Some customers have mentioned that the price is higher than other air mattresses on the market, but they believe it is well worth the cost for the superior comfort and quality. Overall, customers seem satisfied with their purchase of the Luxury Air Mattress and would recommend it to others in search of a high-quality air mattress.'

## SimpleSequentialChain

In [111]:
from langchain.chains import SimpleSequentialChain

In [113]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Describe the features and benefits of the following product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [114]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a social media post to promote the following product description: {product_description}"
)

# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [127]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [128]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Lujoso colchón de aire. Muy cómodo y resistente. Lo recomiendo totalmente para camping o como cama extra en casa. Además, es muy fácil de inflar y desinflar. ¡Una excelente compra!" 

Translation: "Luxury air mattress. Very comfortable and durable. I totally recommend it for camping or as an extra bed at home. In addition, it is very easy to inflate and deflate. An excellent purchase!"
The review describes the air mattress as luxurious, comfortable, and durable, recommending it for camping or as an extra bed at home. The reviewer also mentions that it is easy to inflate and deflate, making it an excellent purchase.

> Finished chain.


'The review describes the air mattress as luxurious, comfortable, and durable, recommending it for camping or as an extra bed at home. The reviewer also mentions that it is easy to inflate and deflate, making it an excellent purchase.'

**Repeat the above twice for different products**

## SequentialChain

In [159]:
from langchain.chains import SequentialChain

In [160]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this product review to English: {Review} "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="translation"
                    )

In [144]:
# Prompt to summarize a review
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following product review in one sentence: {review}"
)

# Chain to summarize
chain_two = LLMChain(
    llm=llm,
    prompt=second_prompt,
    output_key="summary"  # Set the output key name
)

In [161]:
second_prompt = ChatPromptTemplate.from_template(
    "Summrize {translation}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="Summary"
                    )

In [162]:

# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Identify the original language of this summary: {Summary}. "
    "Then, if it's not English, translate it to English:"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="english_translation" 
                      )

In [164]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow up message to this summary: {Summary} and language: {english_translation}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up_message" 
                     )

In [165]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["translation", "Summary", "english_translation", "follow_up_message"],
    verbose=True
)

In [55]:
pip install overall_chain

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement overall_chain (from versions: none)
ERROR: No matching distribution found for overall_chain


In [166]:
review = df.Review[5]
overall_chain(review)

C:\Users\Huawei\AppData\Local\Temp\ipykernel_4108\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translation': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'Summary': 'The person finds the taste of the product mediocre and is disappointed that the foam does not hold. They suspect that the product may be an old batch or counterfeit, as they have had a better experience buying the same item in stores.',
 'english_translation': 'Original language: English\n\nTranslation: N/A',
 'follow_up_message': 'Hello,\n\nThank you for sharing your feedback on the product. We apologize that it did not meet your expectations in terms of taste and foam quality. We can assure you that our products are not old batches or counterfeit, as we take great care in maintaining quality standards. However, we appreciate 

**Repeat the above twice for different products or reviews**

## Router Chain

In [167]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [168]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [169]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [170]:
llm = ChatOpenAI(temperature=0)

In [171]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [172]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [173]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [174]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [175]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [176]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [177]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [178]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation of cells to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential fo

**Repeat the above at least once for different inputs and chains executions - Be creative!**